In [ ]:
# Import các thư viện cần thiết

import pandas as pd
import numpy as np

# Import bộ dữ liệu
df = pd.read_csv('../../data/NASA Near-Earth Objects.csv', index_col=0)
df.index = range(1, len(df) + 1) 

# Xem qua dữ liệu bộ dữ liệu
df.head()